In [1]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops

In [2]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [3]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar100.load_data()

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')

In [4]:
# Преобразуйте метки классов в one_hot формат
y_train = np.eye(CLASSES)[y_train.reshape(-1)]
y_val = np.eye(CLASSES)[y_val.reshape(-1)]

In [5]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [41]:
# Задайте архитектуру модели

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.InputLayer(input_shape=(32, 32, 3)))
model.add(tf.keras.layers.Rescaling(1. / 255))

model.add(tf.keras.layers.SeparableConv2D(20, 3, padding='same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"))

model.add(tf.keras.layers.SeparableConv2D(15, 3, padding='same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"))

model.add(tf.keras.layers.SeparableConv2D(10, 3, padding='same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.LeakyReLU())
#model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"))


model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(63))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(CLASSES))
model.add(tf.keras.layers.Activation('softmax'))

In [42]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")

FLOPs: 0.6604e6


In [43]:
# вывод краткой информации о модели
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_14 (Rescaling)    (None, 32, 32, 3)         0         
                                                                 
 separable_conv2d_42 (Separa  (None, 32, 32, 20)       107       
 bleConv2D)                                                      
                                                                 
 batch_normalization_63 (Bat  (None, 32, 32, 20)       80        
 chNormalization)                                                
                                                                 
 leaky_re_lu_63 (LeakyReLU)  (None, 32, 32, 20)        0         
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 16, 16, 20)       0         
 g2D)                                                            
                                                     

In [44]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=1000, decay_rate=0.5)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [45]:
# обучения модели
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=30
)

Epoch 1/30
391/391 [==============================] - 23s 55ms/step - loss: 3.8288 - accuracy: 0.1154 - val_loss: 4.2733 - val_accuracy: 0.0838
Epoch 2/30
391/391 [==============================] - 21s 54ms/step - loss: 3.4168 - accuracy: 0.1769 - val_loss: 5.3254 - val_accuracy: 0.0868
Epoch 3/30
391/391 [==============================] - 21s 55ms/step - loss: 3.2679 - accuracy: 0.2078 - val_loss: 3.1210 - val_accuracy: 0.2414
Epoch 4/30
391/391 [==============================] - 21s 55ms/step - loss: 3.1781 - accuracy: 0.2229 - val_loss: 3.4302 - val_accuracy: 0.1984
Epoch 5/30
391/391 [==============================] - 23s 58ms/step - loss: 3.1070 - accuracy: 0.2355 - val_loss: 2.9234 - val_accuracy: 0.2873
Epoch 6/30
391/391 [==============================] - 21s 55ms/step - loss: 3.0487 - accuracy: 0.2471 - val_loss: 3.3515 - val_accuracy: 0.2141
Epoch 7/30
391/391 [==============================] - 21s 54ms/step - loss: 3.0056 - accuracy: 0.2547 - val_loss: 2.9573 - val_accuracy: